In [1]:
from enum import IntEnum
import time
import jsonrpclib
import subprocess
from subprocess import PIPE, Popen
from threading  import Thread
import sys
import re

from gym import Env, error, spaces, utils
from stable_baselines3 import DQN, PPO, A2C, TD3, SAC
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.env_checker import check_env
import numpy as np

import os
import requests
import shutil
import tempfile
import xml.etree.ElementTree as ET
from io import StringIO, BytesIO

import cv2
import numpy as np
import torch
from PIL import Image

import olympe
from olympe.messages.ardrone3.Piloting import TakeOff, Landing, moveBy, PCMD, moveTo
from olympe.messages.ardrone3.PilotingState import FlyingStateChanged, GpsLocationChanged, moveToChanged
from olympe.enums.ardrone3.PilotingState import FlyingStateChanged_State as FlyingState
from olympe.messages.ardrone3.GPSSettingsState import GPSFixStateChanged
from olympe.messages.gimbal import set_target
from olympe.messages.camera import (
    set_camera_mode,
    set_photo_mode,
    take_photo,
    photo_progress,
)

olympe.log.update_config({
    "loggers": {
    "olympe": {
            "handlers": []
        }
    }
})

In [20]:
class AnafiEnv(Env):
    def __init__(self, DRONE_IP = "10.202.0.1", boundary=10):
        super(AnafiEnv, self).__init__()
        
        self.model = torch.hub.load('yolov5', 'custom', path='weights/best.pt', source='local')
        self.model.conf = 0.7
        self.model.classes = [0]
        
        self.sphinx = jsonrpclib.Server('http://127.0.0.1:8383')
        self.sphinx.SetParam(machine='anafi4k', object='lipobattery/lipobattery', parameter='discharge_speed_factor', value='0')
        
        self.drone = olympe.Drone(DRONE_IP)
        self.drone.connect()
        self.setup_photo_single_mode()
        self.takeoff()
        self.home = self.drone.get_state(GpsLocationChanged)
        self.drone.start_piloting()
#         self.drone(PCMD(1, 0, 0, 0, -3, 0) >> FlyingStateChanged(state="hovering", _timeout=5)).wait()
#         time.sleep(2)
#         self.drone.piloting_pcmd(0, 0, 0, -3, 1)
#         self.drone.piloting_pcmd(0, 0, 0, -2, 0.5) # roll, pitch, yaw, gaz
#         time.sleep(1.5)
        self.drone(
                                moveBy(0, 0, -3, 0)  # (forward-backward, right-left, down-up, clockwise-anticlockwise)
                                >> FlyingStateChanged(state="hovering", _timeout=5)
                                ).wait().success()
        self.agent_coord = np.zeros(3)
        
        self.action_space = spaces.Box(low=-1, high=1, shape=(3,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-1, high=1, shape=(3,), dtype=np.float32)
        
        ON_POSIX = 'posix' in sys.builtin_module_names
        cmd = "parrot-gz topic -e /gazebo/default/pose/info | grep -A 5 'name: \"anafi4k\"'"
        p = Popen(cmd, stdout=PIPE, bufsize=1, close_fds=ON_POSIX, shell=True)
        self.odom_thread = Thread(target=self.set_current_agent_coord, args=[p.stdout], daemon=True)
        self.odom_thread.start()
        
        self.step_count = 0
        self.boundary = boundary
    
    def setup_photo_single_mode(self):
        self.drone(set_camera_mode(cam_id=0, value="photo")).wait()
        # For the file_format: jpeg is the only available option
        # dng is not supported in burst mode
        self.drone(
            set_photo_mode(
                cam_id=0,
                mode="single",
                format="rectilinear",
                file_format="jpeg",
                burst="burst_14_over_1s",
                bracketing="preset_1ev",
                capture_interval=0.0,
                )
            ).wait()
        
        self.drone(
            set_target(
                gimbal_id=0,
                control_mode="position",
                yaw_frame_of_reference="none",   # None instead of absolute
                yaw=0.0,
                pitch_frame_of_reference="absolute",
                pitch=-90.0,
                roll_frame_of_reference="none",     # None instead of absolute
                roll=0.0,
                )
            ).wait()
        
    def take_photo_single(self, download_dir):
        # Drone IP
        ANAFI_IP = "10.202.0.1"

        # Drone web server URL
        ANAFI_URL = "http://{}/".format(ANAFI_IP)

        # Drone media web API URL
        ANAFI_MEDIA_API_URL = ANAFI_URL + "api/v1/media/medias/"

        self.drone(PCMD(1, 0, 0, 0, 0, 0) >> FlyingStateChanged(state="hovering", _timeout=5)).wait()
        photo_saved = self.drone(photo_progress(result="photo_saved", _policy="wait"))
        self.drone(take_photo(cam_id=0)).wait()
        if not photo_saved.wait().success():
            print("Photos not saved")
        media_id = photo_saved.received_events().last().args["media_id"]

        # download the photos associated with this media id
        media_info_response = requests.get(ANAFI_MEDIA_API_URL + media_id)
        media_info_response.raise_for_status()
        for resource in media_info_response.json()["resources"]:
            image_response = requests.get(ANAFI_URL + resource["url"], stream=True)
#             download_path = os.path.join(download_dir, resource["resource_id"])
            image_response.raise_for_status()
            img = Image.open(BytesIO(image_response.content))
            result = self.model(img)
            result.print()
            result.show()
            print(result.xyxy[0].shape[0])
#             with open(download_path, "wb") as image_file:
#                 shutil.copyfileobj(image_response.raw, image_file)
        
        # DELETE THE FILE AFTER DOWNLOADING
    
    def get_current_agent_coord(self):
        return self.agent_coord
    
    def set_current_agent_coord(self, output):
        for line in iter(output.readline, b''):
            line = str(line)
            
            if "x:" in line:
                x = re.findall(r"[-+]?\d*\.\d+|\d+", line)[0]
                self.agent_coord[0] = float(x)
            elif "y:" in line:
                y = re.findall(r"[-+]?\d*\.\d+|\d+", line)[0]
                self.agent_coord[1] = float(y)
            elif "z:" in line:
                z = re.findall(r"[-+]?\d*\.\d+|\d+", line)[0]
                self.agent_coord[2] = float(z)
            
    def step(self, action):
        
        done = False
        reward = 0
        
        max_x, max_y, max_z, min_z = 10, 10, 8, 3
        
        x, y, z = self.agent_coord
        
        # change the pitch
        if x > max_x:
            action[1] = -1
        elif x < -max_x:
            action[1] = 1
        # change the roll
        elif y > max_y:
            action[0] = 1
        elif y < -max_y:
            action[0] = -1
        # change the throttle
        elif z > max_z:
            action [2] = -0.5
        elif z < min_z:
            action [2] = 0.5
        
        self.take_action(action)
        
        self.step_count += 1
        
        obs = self.agent_coord / self.boundary
            
        if (self.agent_coord >= self.boundary).any() or self.step_count >= 10:
            done = True
            self.step_count = 0
        
        return obs, reward, done, {}
        
    def take_action(self, action):
        action = action*100
        self.drone.piloting_pcmd(action[0], action[1], 0, action[2], 0.5) # roll, pitch, yaw, gaz
        time.sleep(0.5)
    
    def takeoff(self):
        assert self.drone(
                    FlyingStateChanged(state="hovering", _policy="check")
                    | (
                        GPSFixStateChanged(fixed=1, _timeout=10)
                        >> (
                            TakeOff(_no_expect=True)
                            & FlyingStateChanged(state="hovering", _policy="wait", _timeout=5)
                        )
                    )
                ).wait().success()
        
    def reset(self):
        latitude, longitude, altitude = self.home["latitude"], self.home["longitude"], self.home["altitude"]
        
        resetAction = self.drone(
            moveTo(latitude,  longitude, altitude, "HEADING_DURING", 90.0)
            >> moveToChanged(status="DONE", _timeout=15)
        ).wait()
    
        return self.agent_coord / self.boundary
    
    def render(self, mode='human'):
        pass
    
    def close(self):
        self.drone.stop_piloting()
        self.drone(PCMD(1, 0, 0, 0, 0, 0) >> FlyingStateChanged(state="hovering", _timeout=5)).wait()
        assert self.drone(Landing() >> FlyingStateChanged(state="landed")).wait().success()
        self.drone.disconnect()

In [21]:
env = AnafiEnv()
download_dir = tempfile.mkdtemp()
for _ in range(1):
    for _ in range(5):
        action = env.action_space.sample()
        env.step(action)
    print("taking a picture")
    env.take_photo_single(download_dir)
env.reset()
env.close()

YOLOv5 🚀 v5.0-392-gb894e69 torch 1.9.0+cu111 CUDA:0 (GeForce RTX 3060 Laptop GPU, 5921.5625MB)

Fusing layers... 
Model Summary: 308 layers, 21037638 parameters, 0 gradients
Adding AutoShape... 
Creating pomp loop
device callbacks have been added to arsdk_ctrl
Creating pomp timer
Creating pomp loop
Creating pomp timer
New device has been detected: 'ANAFI-0000000'
Connecting to device: ANAFI-0000000
Connection in progress...
Connected to device: ANAFI-0000000
{'json': {'c2d_port': 2233,
          'c2d_update_port': 51,
          'c2d_user_port': 21,
          'proto_v': 3,
          'qos_mode': 0,
          'status': 0}}
Creating pomp loop
Command interface has been created: itf=None
Creating pomp loop
rth.ending_behavior(ending_behavior=ending_behavior.hovering)
Creating pomp loop
common.Common.AllStates() has been sent to the device
common.CommonState.BatteryStateChanged(percent=99)
common.ARLibsVersionsState.DeviceLibARCommandsVersion(version='4.0.0.4')
common.CommonState.BootId(boot

camera.recording_mode(cam_id=0, mode=recording_mode.standard, resolution=resolution.res_uhd_4k, framerate=framerate.fps_30, hyperlapse=hyperlapse_value.ratio_15, bitrate=10000000, list_flags='')
camera.white_balance(cam_id=0, mode=white_balance_mode.automatic, temperature=white_balance_temperature.t_1500, lock=state.inactive, list_flags='')
camera.exposure_settings(cam_id=0, mode=exposure_mode.automatic, manual_shutter_speed=shutter_speed.shutter_1_over_10000, manual_shutter_speed_capabilities='shutter_1_over_10000|shutter_1_over_8000|shutter_1_over_6400|shutter_1_over_5000|shutter_1_over_4000|shutter_1_over_3200|shutter_1_over_2500|shutter_1_over_2000|shutter_1_over_1600|shutter_1_over_1250|shutter_1_over_1000|shutter_1_over_800|shutter_1_over_640|shutter_1_over_500|shutter_1_over_400|shutter_1_over_320|shutter_1_over_240|shutter_1_over_200|shutter_1_over_160|shutter_1_over_120|shutter_1_over_100|shutter_1_over_80|shutter_1_over_60|shutter_1_over_50|shutter_1_over_40|shutter_1_over_30

rth.auto_trigger_mode(mode=auto_trigger_mode.on)
rth.state(state=state.unavailable, reason=state_reason.disabled)
ardrone3.PilotingSettingsState.MotionDetection(enabled=0)
rth.takeoff_location(latitude=48.87889915518789, longitude=2.3677815763152203, altitude=2.0, fixed_before_takeoff=1)
rth.preferred_home_type(type=home_type.takeoff)
ardrone3.GPSSettingsState.HomeTypeChanged(type=HomeTypeChanged_Type.TAKEOFF)
common.SettingsState.ProductNameChanged(name='Parrot Drone')
wifi.supported_security_types(types='open|wpa2')
wifi.security_changed(type=security_type.open, key='', key_type=security_key_type.plain)
wifi.ap_channel_changed(type=selection_type.auto_all, band=-1, channel=255)
wifi.country_changed(selection_mode=country_selection.auto, code='FR')
wifi.environment_changed(environment=environment.outdoor)
gimbal.relative_attitude_bounds(gimbal_id=0, min_yaw=0.0, max_yaw=0.0, min_pitch=-135.0, max_pitch=105.0, min_roll=-38.0, max_roll=38.0, list_flags='')
gimbal.absolute_attitude_bound

camera.photo_mode(cam_id=0, mode=photo_mode.single, format=photo_format.rectilinear, file_format=photo_file_format.jpeg, burst=burst_value.burst_14_over_1s, bracketing=bracketing_preset.preset_1ev, capture_interval=0.0, list_flags='')
gimbal.set_target(0, <control_mode.position: 0>, <frame_of_reference.none: 0>, 0.0, <frame_of_reference.absolute: 2>, -90.0, <frame_of_reference.none: 0>, 0.0) has been sent to the device
ardrone3.Piloting.TakeOff() has been sent to the device
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.motor_ramping)
common.RunState.RunIdChanged(runId='CE4E47')
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.takingoff)
follow_me.mode_info(mode=mode.look_at, missing_requirements='drone_calibrated|drone_gps_good_accuracy|target_gps_good_accuracy|target_barometer_ok|drone_far_enough|drone_high_enough|target_good_speed|drone_close_enough', improvements='drone_calibrated|drone_gps_good_accuracy|target_gps_good_accuracy|tar

taking a picture


media_event(name=media_created, media_id=10000012, photo_mode=SINGLE)
camera.photo_state(cam_id=0, available=availability.available, state=state.active, list_flags='')
camera.photo_progress(cam_id=0, result=photo_result.taking_photo, photo_count=0, media_id='', list_flags='')
camera.photo_progress(cam_id=0, result=photo_result.photo_taken, photo_count=1, media_id='', list_flags='')
camera.photo_progress(cam_id=0, result=photo_result.photo_saved, photo_count=0, media_id='10000012', list_flags='')
camera.photo_state(cam_id=0, available=availability.available, state=state.inactive, list_flags='')
mediastore.counters(video_media_count=0, photo_media_count=12, video_resource_count=0, photo_resource_count=12)
image 1/1: 720x1280 4 targets
Speed: 11.0ms pre-process, 20.9ms inference, 2.5ms NMS per image at shape (1, 3, 384, 640)
ardrone3.Piloting.moveTo(48.87889882418161, 2.367781495536632, 0.7583574056625366, <MoveTo_Orientation_mode.HEADING_DURING: 3>, 90.0) has been sent to the device
ardr

4


follow_me.mode_info(mode=mode.look_at, missing_requirements='drone_calibrated|drone_gps_good_accuracy|target_gps_good_accuracy|target_barometer_ok|drone_far_enough|drone_high_enough|target_good_speed|drone_close_enough', improvements='drone_calibrated|drone_gps_good_accuracy|target_gps_good_accuracy|target_barometer_ok|drone_far_enough|drone_high_enough|image_detection|target_good_speed|drone_close_enough', list_flags='')
follow_me.mode_info(mode=mode.geographic, missing_requirements='drone_calibrated|drone_gps_good_accuracy|target_barometer_ok|drone_far_enough|drone_high_enough|target_good_speed|drone_close_enough', improvements='drone_calibrated|drone_gps_good_accuracy|target_gps_good_accuracy|target_barometer_ok|drone_far_enough|drone_high_enough|image_detection|target_good_speed|drone_close_enough', list_flags='')
follow_me.mode_info(mode=mode.relative, missing_requirements='drone_calibrated|drone_gps_good_accuracy|target_barometer_ok|drone_far_enough|drone_high_enough|target_good_

follow_me.mode_info(mode=mode.leash, missing_requirements='drone_calibrated|drone_gps_good_accuracy|target_barometer_ok|drone_far_enough|drone_high_enough|target_good_speed|drone_close_enough', improvements='drone_calibrated|drone_gps_good_accuracy|target_gps_good_accuracy|target_barometer_ok|drone_far_enough|drone_high_enough|image_detection|target_good_speed|drone_close_enough', list_flags='')
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.landed)
we are not disconnected yet
Disconnected from device: ANAFI-0000000
disconnected from device: b'10.202.0.1'
Disconnection with the device OK. IP: b'10.202.0.1'
olympe.media shutdown
Pomp loop has been destroyed
Pomp loop has been destroyed
pdraw callbacks thread loop stopped


In [ ]:
drone = olympe.Drone("10.202.0.1")
drone.connect()
drone(TakeOff()).wait()
drone.start_piloting()

In [ ]:
drone.piloting_pcmd(0, 0, 0, 100, 0.5) # y, x, NULL, z

In [1]:
import cv2
import torch
from PIL import Image

In [2]:
model = torch.hub.load('yolov5', 'custom', path='weights/best.pt', source='local')

YOLOv5 🚀 v5.0-392-gb894e69 torch 1.9.0+cu111 CUDA:0 (GeForce RTX 3060 Laptop GPU, 5921.5625MB)

Fusing layers... 
Model Summary: 308 layers, 21037638 parameters, 0 gradients
Adding AutoShape... 


In [3]:
img = Image.open('images/100001270127.JPG')

In [4]:
model.conf = 0.7
model.classes = [0]
result = model(img)

In [5]:
result.print()

image 1/1: 720x1280 18 targets
Speed: 26.0ms pre-process, 41.5ms inference, 3.7ms NMS per image at shape (1, 3, 384, 640)


In [6]:
result.xyxy[0].shape[0]

18

In [7]:
result.show()

In [11]:
type(img)

PIL.JpegImagePlugin.JpegImageFile